In [ ]:
import glob
import pandas as pd
import uproot
import uproot_methods
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LightSource
import matplotlib.colors as mcolors

%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable


plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import interpolate

import itertools

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dir = r"../PYTHON_graphs/OUTPUTS/standard_graphs/"
dirN2 = r"../PYTHON_graphs/OUTPUTS/N2_calibration/"

allfiles = (glob.glob("../PYTHON_graphs/DATA/Standard_graphs/*.root"))

cmap_viridis = mpl.cm.get_cmap('viridis')

In [ ]:
allfiles

In [ ]:
for el in allfiles:
    if "elec_linearity" in el:
        elec_lin = uproot.open(el)
        x_el_lin,y_el_lin,counts_el_lin=fugi.import_TH2Dgeneric(elec_lin,elec_lin.keys()[0])
    elif "pipico_momcon_shrt" in el:
        pipico_n2 = uproot.open(el)
    elif "recoil_sum_w" in el:
        recoil_sum_w = uproot.open(el)
        w_layer_rec,tsum_rec,counts_rec_sum=fugi.import_TH2Dgeneric(recoil_sum_w,recoil_sum_w.keys()[0])
    elif "rec_xy" in el:
        recoil_xy = uproot.open(el)
        x_rec,y_rec,counts_rec_xy=fugi.import_TH2Dgeneric(recoil_xy,recoil_xy.keys()[0])
    elif "pipico_momcon" in el:
        pipico_n2_momcon = uproot.open(el)
        tof1,tof2,counts_pipico=fugi.import_TH2Dgeneric(pipico_n2_momcon,pipico_n2_momcon.keys()[0])
    elif "elec_sum_w" in el:
        elec_sum_w = uproot.open(el)
        w_layer,tsum,counts_el_sum=fugi.import_TH2Dgeneric(elec_sum_w,elec_sum_w.keys()[0])
    elif "elec_xy" in el:
        elec_xy = uproot.open(el)
        x_el,y_el,counts_el_xy=fugi.import_TH2Dgeneric(elec_xy,elec_xy.keys()[0])
    elif "TFMOX_pipico_short" in el:
        tfmox_pipico = uproot.open(el)
        tmox_tof1,tmox_tof2,counts_pipico_tmox=fugi.import_TH2Dgeneric(tfmox_pipico,tfmox_pipico.keys()[0])
    elif "PIPICO_CH9" in el:
        tfmox_pipico_ch9 = uproot.open(el)
        tof9_1,tof9_2,couts_pipico_9=fugi.import_TH2Dgeneric(tfmox_pipico_ch9,tfmox_pipico_ch9.keys()[0])

In [ ]:
# fileSCR_11[loc11].keys()

In [ ]:
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/", "ID4_mol_e0_valid/EN_gate/"]
ID=str(IDs[3]).split("_")[0]

# dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

fileSCR_12 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
path12 = "angular_distr_el/CH12/"
loc12=path12+IDs[1]

fileSCR_11 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
path11 = "angular_distr_el/CH11/"
loc11=path11+IDs[3]

fileSCR_9 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
path9 = "angular_distr_el/CH9/"
loc9=path9+IDs[3]

fileSCR_14 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_ALLCH_diatomic.root")
path14 = "angular_distr_el/CH14/"
loc14=path14+IDs[0]

pz,px,counts_newton11=fugi.import_TH2Dgeneric(fileSCR_11,loc11+"/Newton plot")
pz,px,counts_newton9=fugi.import_TH2Dgeneric(fileSCR_9,loc9+"/Newton plot")
# check_ion[2]_neutral_xy
# KER, p_sumx vs p_sumz, p_relx vs p_relz

tof14_1,tof14_2,couts_pipico_14=fugi.import_TH2Dgeneric(fileSCR_14,loc14+"/PIPICO")
tof11_1,tof11_2,couts_pipico_11=fugi.import_TH2Dgeneric(fileSCR_11,"test/PIPICO")

In [ ]:
newton12_all=[]; newton11_all=[]; newton9_all=[]
for i,el in enumerate(IDs):
    if i == 1:
        loc12=path12+el
        dummy1,dummy2,plot=fugi.import_TH2Dgeneric(fileSCR_12,loc12+"/Newton plot")
        newton12_all.append(plot)
    else:
        loc11=path11+el
        dummy1,dummy2,plot=fugi.import_TH2Dgeneric(fileSCR_11,loc11+"/Newton plot")
        newton11_all.append(plot)
    if i <=3:
        loc9=path9+el
        dummy1,dummy2,plot=fugi.import_TH2Dgeneric(fileSCR_9,loc9+"/Newton plot")
        newton9_all.append(plot)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))
counts_el_sum_red=np.ma.masked_array(counts_el_sum,counts_el_sum<3)

# cs=ax.contourf(w_layer,tsum,counts_el_sum_red.T, levels=40)
cs=ax.pcolormesh(w_layer,tsum,counts_el_sum_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-1,1)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_sum_red.min(),counts_el_sum_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"el_sum_wlayer.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
# d = np.linspace(-0.5, 0.5, 50)
counts_el_sum_time=np.sum(counts_el_sum,axis=0)
counts_el_sum_time_red=np.ma.masked_array(counts_el_sum_time,counts_el_sum_time<500).compressed()
tsum_red=np.ma.masked_array(tsum,counts_el_sum_time<500).compressed()

mean = sum(tsum_red*counts_el_sum_time_red)/sum(counts_el_sum_time_red)
sigma = np.sqrt(sum(counts_el_sum_time_red*(tsum_red-mean)**2)/sum(counts_el_sum_time_red))

def gaus_fit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus_fit,tsum_red,counts_el_sum_time_red,p0=[max(counts_el_sum_time_red), mean, sigma])

FWHM=2*np.sqrt(2*np.log(2))*sigma
fit_info = "\u00B5 = %.2f ns \n$\u03c3_{std}$ = %.2f \nFWHM = %.2f ps" % (mean, sigma, FWHM*10**3)
# fit_info = (f"Gauss fit: \u00B5 = {mu:+.2f}, \u00b2 std = {std:+.2f}")

fig,ax= plt.subplots(1, figsize=(12, 10))
ax.set_xlabel('time sum [ns]')
ax.set_ylabel('counts')
ax.set_xlim([-0.8, 1])
ax.ticklabel_format(axis="y",style="sci",scilimits=(0,1))

ax.set_box_aspect(1)

ax.plot(tsum_red,counts_el_sum_time_red, color=cmap_viridis(0),alpha = 0.6, linewidth=3)
ax.plot(tsum_red,gaus_fit(tsum_red,*popt), color=cmap_viridis(0.3), ls="--",linewidth=2, label="Gaussian fit")
ax.plot([-FWHM/2*0.45,FWHM/2*1.05], [counts_el_sum_time_red.max()/2,counts_el_sum_time_red.max()/2], color=cmap_viridis(0.3), linewidth=2., ls='--', alpha=0.5)
plt.plot([], [], ' ', label=fit_info)

ax.legend(loc="best",fontsize="small")
ax.grid()


plt.plot()
fig.savefig(dir+"el_sum_wlayer_proj.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))
counts_rec_sum_red=np.ma.masked_array(counts_rec_sum,counts_rec_sum<10)

# cs=ax.contourf(w_layer,tsum,counts_el_sum_red.T, levels=40)
cs=ax.pcolormesh(w_layer_rec,tsum_rec,counts_rec_sum_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-2,2)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_rec_sum_red.min(),counts_rec_sum_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"rec_sum_wlayer.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
# d = np.linspace(-0.5, 0.5, 50)
counts_rec_sum_time=np.sum(counts_rec_sum,axis=0)

counts_rec_sum_time_red1=np.ma.masked_array(counts_rec_sum_time,counts_rec_sum_time<500)
counts_rec_sum_time_red=np.ma.masked_array(counts_rec_sum_time,tsum_rec>0.4).compressed()
tsum_rec_red1=np.ma.masked_array(tsum_rec,counts_rec_sum_time<500)
tsum_rec_red=np.ma.masked_array(tsum_rec,tsum_rec>0.4).compressed()

mean = sum(tsum_rec_red*counts_rec_sum_time_red)/sum(counts_rec_sum_time_red)
sigma = np.sqrt(sum(counts_rec_sum_time_red*(tsum_rec_red-mean)**2)/sum(counts_rec_sum_time_red))

def gaus_fit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus_fit,tsum_rec_red,counts_rec_sum_time_red,p0=[max(counts_rec_sum_time_red), mean, sigma])

FWHM=2*np.sqrt(2*np.log(2))*sigma
fit_info = "\u00B5 = %.2f ns \n$\u03c3_{std}$ = %.2f \nFWHM = %.2f ps" % (mean, sigma, FWHM*10**3)
# fit_info = (f"Gauss fit: \u00B5 = {mu:+.2f}, \u00b2 std = {std:+.2f}")

fig,ax= plt.subplots(1, figsize=(12, 10))
ax.set_xlabel('time sum [ns]')
ax.set_ylabel('counts')
ax.set_xlim([-0.8, 1])
ax.ticklabel_format(axis="y",style="sci",scilimits=(0,1))

ax.set_box_aspect(1)

ax.plot(tsum_rec_red,counts_rec_sum_time_red, color=cmap_viridis(0),alpha = 0.6, linewidth=3)
ax.plot(tsum_rec_red,gaus_fit(tsum_rec_red,*popt), color=cmap_viridis(0.3), ls="--",linewidth=2, label="Gaussian fit")
ax.plot([-FWHM/2*0.45,FWHM/2*1.05], [counts_rec_sum_time_red.max()/2,counts_rec_sum_time_red.max()/2], color=cmap_viridis(0.3), linewidth=2., ls='--', alpha=0.5)
plt.plot([], [], ' ', label=fit_info)

ax.legend(loc="best",fontsize="small")
ax.grid()


plt.plot()
fig.savefig(dir+"rec_sum_wlayer_proj.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
xy_el=np.array(list(itertools.product(x_el, y_el)))
X_el=np.array([col[0] for col in xy_el])
Y_el=np.array([col[1] for col in xy_el])

xy_rec=np.array(list(itertools.product(x_rec, y_rec)))
X_rec=np.array([col[0] for col in xy_rec])
Y_rec=np.array([col[1] for col in xy_rec])

In [ ]:
#3D MFPAD
counts_el_xy_new=np.ma.masked_where(np.sqrt(X_el**2+Y_el**2)<25,counts_el_xy.reshape(-1)).reshape(len(x_el),len(y_el))
counts_el_xy_new_red=np.ma.masked_array(counts_el_xy_new, counts_el_xy_new<=5)
norm=mcolors.LogNorm(counts_el_xy_new_red.min(),counts_el_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el,y_el,counts_el_xy_new_red.T, levels=20)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud')
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 20], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_xy_new_red.min(),counts_el_xy_new_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MCP_el_xy_red.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_el_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD

counts_rec_xy_new=np.ma.masked_where(np.sqrt(X_rec**2+Y_rec**2)<25,counts_rec_xy.reshape(-1)).reshape(len(x_rec),len(y_rec))
counts_rec_xy_new_red=np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5)
norm=mcolors.LogNorm(counts_rec_xy_new_red.min(),counts_rec_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_rec.max()/y_rec.max())
# cs=ax.contourf(x_rec,y_rec,counts_rec_xy_new_red.T, levels=20)
# cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud')
cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 19], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_rec_xy_new_red.min(),counts_rec_xy_new_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MCP_rec_xy_red.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_rec_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
counts_el_xy_red=np.ma.masked_array(counts_el_xy,counts_el_xy<1)
norm=mcolors.LogNorm(counts_el_xy_red.min(),counts_el_xy_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())

# cs=ax.contourf(x_el,y_el,counts_el_xy_red.T, levels=20)
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud', norm=norm)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_xy_red.min(),counts_el_xy_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"el_mom_ex.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_mom_ex.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
x_el_lin_r=fugi.remap(x_el_lin,-45,45)
y_el_lin_r=fugi.remap(y_el_lin,-45,45)
counts_el_lin_red=np.ma.masked_array(counts_el_lin,counts_el_lin<0.00001)
norm=mcolors.LogNorm(counts_el_lin_red.min(),counts_el_lin_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el_lin_r,y_el_lin_r,counts_el_lin_red, levels=40)
# cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red, shading='gouraud')
cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([counts_el_lin_red.min(),counts_el_lin_red.max()])
# cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('deviation from linearity')

fig.savefig(dir+"MCP_el_linearity.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_el_linearity.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tof1.max()/tof2.max())
counts_pipico_red=np.ma.masked_array(counts_pipico,counts_pipico<1)
# ax.set_title("fish fillet y-axis")

cs=ax.pcolormesh(tof1,tof2,counts_pipico_red.T, shading='gouraud')

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(0,5000)
ax.set_ylim(0,5000)
ax.set_aspect(aspect)

axins = ax.inset_axes([0.55, 0.06, 0.42, 0.42])
axins.pcolormesh(tof1,tof2,np.ma.masked_array(counts_pipico.T,counts_pipico.T<1), shading='gouraud')
axins.set_xlim(2800,3100)
axins.set_ylim(2900,3200)
axins.tick_params(axis='x', labelsize=12)
axins.tick_params(axis='y', labelsize=12)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_pipico_red.min(),counts_pipico_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dirN2+"N2_pipico_insert.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"N2_pipico_insert.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tmox_tof1.max()/tmox_tof2.max())
counts_pipico_tmox_red=np.ma.masked_array(counts_pipico_tmox,counts_pipico_tmox<2)
norm=mcolors.LogNorm(counts_pipico_tmox_red.min(),counts_pipico_tmox_red.max())

# ax.set_title("fish fillet y-axis")

cs=ax.pcolormesh(tmox_tof1,tmox_tof2,counts_pipico_tmox_red.T, shading='gouraud', norm=norm)

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(0,7500)
ax.set_ylim(0,7500)
ax.set_aspect(aspect)

rect1 = patches.Rectangle((2500, 6400), 3000, 500, linewidth=2, ls= "--", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect1)
rect2 = patches.Rectangle((2500, 5400), 3000, 500, linewidth=2, ls= ":", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect2)
rect3 = patches.Rectangle((2500, 4200), 2100, 500, linewidth=2, ls= "-.", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect3)
rect4 = patches.Rectangle((2650, 7000), 600, -3000, linewidth=2, ls= "--", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(rect4)
rect5 = patches.Rectangle((3800, 7000), 700, -3000, linewidth=2, ls= ":", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(rect5)
circ1 = patches.Circle((5150, 6650), radius=300, linewidth=2, ls= ":", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(circ1)

axins = ax.inset_axes([0., 0.65, 1., 1.])
axins.pcolormesh(tmox_tof1,tmox_tof2,np.ma.masked_array(counts_pipico_tmox.T,counts_pipico_tmox.T<2), shading='gouraud',norm=norm )
axins.set_xlim(2500,5500)
axins.set_ylim(6400,6900)
axins.tick_params(axis='x', labelsize=12)
axins.tick_params(axis='y', labelsize=12)
axins.set_aspect("equal")

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_pipico_tmox_red.min(),counts_pipico_tmox_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"TMOX_pipico_insert.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"TMOX_pipico_insert.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_11_red=np.ma.masked_array(couts_pipico_11,couts_pipico_11<2)

cs=ax.pcolormesh(tof11_1,tof11_2,couts_pipico_11_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(3850,4500)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([couts_pipico_11_red.min(),couts_pipico_11_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"TFMOX_PIPICO_ch11.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_9_red=np.ma.masked_array(couts_pipico_9,couts_pipico_9<2)
# norm=mcolors.LogNorm(couts_pipico_9_red.min(),couts_pipico_9_red.max())

cs=ax.pcolormesh(tof9_1,tof9_2,couts_pipico_9_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
# ax.set_xlim(3900,4500)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([couts_pipico_9_red.min(),couts_pipico_9_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"TFMOX_PIPICO_ch9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_14_red=np.ma.masked_array(couts_pipico_14,couts_pipico_14<2)

cs=ax.pcolormesh(tof14_1,tof14_2,couts_pipico_14_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(4900,5400)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([couts_pipico_14_red.min(),couts_pipico_14_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"TFMOX_PIPICO_ch14.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[1]).split("_")[0]
counts_newton12_red=np.ma.masked_array(newton12_all[0],newton12_all[0]<1)

cs=ax.pcolormesh(pz,px,counts_newton12_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('$p_{x}$ [a.u.]')
ax.set_ylabel('$p_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1, color="r", alpha=0.6)
ax.add_patch(arrow1)
text=ax.text(0.55,0.15, "$CH_{2}O^{+}$") #,fontweight="bold")
text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_newton12_red.min(),counts_newton12_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"Netwon_"+ID_local+"_ch12.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[0]).split("_")[0]
counts_newton11_red=np.ma.masked_array(newton11_all[0],newton11_all[0]<2)

cs=ax.pcolormesh(pz,px,counts_newton11_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('$p_{x}$ [a.u.]')
ax.set_ylabel('$p_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1, color="r", alpha=0.6)
ax.add_patch(arrow1)
if ID_local == "ID":
        text=ax.text(0.55,0.15, "$C_{2}H_{i=1,2,3}^{+}$") #,fontweight="bold")
if ID_local == "ID2":
        text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
if ID_local == "ID3":
        text=ax.text(0.85,0.15, "$C_{2}H_{2}^{+}$") #,fontweight="bold")
if ID_local == "ID4":
        text=ax.text(0.85,0.15, "$C_{2}H_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('$p_{x}$ [a.u.]')

text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_newton11_red.min(),counts_newton11_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"Netwon_"+ID_local+"_ch11.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[0]).split("_")[0]
counts_newton9_red=np.ma.masked_array(newton9_all[0],newton9_all[0]<2)

counts_newton9_red=np.ma.masked_array(counts_newton9,counts_newton9<10)

cs=ax.pcolormesh(pz,px,counts_newton9_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('$p_{x}$ [a.u.]')
ax.set_ylabel('$p_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
ax.add_patch(arrow1)
if ID_local == "ID":
    text=ax.text(0.55,0.15, "$CH_{i=0,1,2,3}^{+}$") #,fontweight="bold")
if ID_local == "ID1":
    text=ax.text(0.85,0.15, "$CH^{+}$") #,fontweight="bold")
if ID_local == "ID2":
    text=ax.text(0.85,0.15, "$CH_{2}^{+}$") #,fontweight="bold")
if ID_local == "ID3":
    text=ax.text(0.85,0.15, "$CH_{3}^{+}$") #,fontweight="bold")

text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_newton9_red.min(),counts_newton9_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"Netwon_"+ID_local+"_ch9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,axes= plt.subplots(4,1, figsize=(7, 25), sharex=True, constrained_layout=True)

for i,ax in enumerate(axes.ravel()):
    # el=np.ma.masked_array(newton11_all[i+1],newton11_all[i+1]<5)
    el=np.ma.masked_array(newton11_all[i],newton11_all[i]<1)
    cs=ax.pcolormesh(pz,px,el.T, shading='gouraud')

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.set_ylabel('$p_{y}$ [a.u.]')
    ax.set_xlim(-2,1.5)
    ax.set_ylim(-1.5,1.5)

    arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
    ax.add_patch(arrow1)

    if i == 0:
        text=ax.text(0.4,0.15, "$C_{2}H_{i=1,2,3}^{+}$") #,fontweight="bold")
    if i == 1:
        text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
    if i == 2:
        text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
    if i == 3:
        text=ax.text(0.85,0.15, "$C_{2}H_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('$p_{x}$ [a.u.]')
    
    text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, location='top', ax=axes[0])
cbar.set_ticks([el.min(),el.max()])
cbar.set_ticklabels(["min","max"])
# cbar.ax.set_ylabel('normalized counts', rotation="horizontal", ha='right')

fig.savefig(dir+"Netwon_all_CH11.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
import matplotlib.patheffects as PathEffects

In [ ]:
#sum el w-layer
fig,axes= plt.subplots(4,1, figsize=(7, 25), sharex=True, constrained_layout=True)

for i,ax in enumerate(axes.ravel()):
    # el=np.ma.masked_array(newton9_all[i+1],newton9_all[i+1]<1)
    el=np.ma.masked_array(newton9_all[i],newton9_all[i]<1)
    cs=ax.pcolormesh(pz,px,el.T, shading='gouraud')

    ax.set_ylabel('$p_{y}$ [a.u.]')
    ax.set_xlim(-2,1.5)
    ax.set_ylim(-1.5,1.5)

    arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
    ax.add_patch(arrow1)

    if i == 0:
        text=ax.text(0.4,0.15, "$CH_{i=0,1,2,3}^{+}$") #,fontweight="bold")
    if i == 1:
        text=ax.text(0.85,0.15, "$CH^{+}$") #,fontweight="bold")
    if i == 2:
        text=ax.text(0.85,0.15, "$CH_{2}^{+}$") #,fontweight="bold")
    if i == 3:
        text=ax.text(0.85,0.15, "$CH_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('$p_{x}$ [a.u.]')
    
    text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, location='top', ax=axes[0])
cbar.set_ticks([el.min(),el.max()])
cbar.set_ticklabels(["min","max"])
# cbar.ax.set_ylabel('normalized counts', rotation="horizontal", ha='right')

fig.savefig(dir+"Netwon_all_CH9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#TFMOX 
#cos theta vs energy CH14
#momentum sum/rel